<a href="https://colab.research.google.com/github/mohammed3200/Adaan/blob/main/CNN_and_Apriori_Association_Rules_with_Network_Intrusion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from mlxtend.frequent_patterns import apriori, association_rules

In [ ]:
# Step 2: Load datasets
train_data = pd.read_csv('/content/drive/MyDrive/content/Train_data.csv')
test_data = pd.read_csv('/content/drive/MyDrive/content/Test_data.csv')

In [ ]:
# Step 3: Data Preprocessing
def preprocess_data(df):
    # Convert categorical columns to one-hot encoding
    df_encoded = pd.get_dummies(df, drop_first=True)
    return df_encoded

train_data_encoded = preprocess_data(train_data)
test_data_encoded = preprocess_data(test_data)

In [ ]:
# Step 4: Generate Association Rules
def generate_association_rules(data):
    frequent_itemsets = apriori(data, min_support=0.1, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
    return rules

rules = generate_association_rules(train_data_encoded)

In [ ]:
# Step 5: Prepare features based on the rules
def prepare_features_with_rules(data, rules):
    # Create a list of features from the rules' antecedents
    features = rules['antecedents'].apply(lambda x: list(x)).explode().value_counts().index.tolist()
    X_features = np.zeros((data.shape[0], len(features)))

    for i, row in enumerate(data.values):
        for feature in features:
            if feature in row:
                X_features[i, features.index(feature)] = 1

    return X_features

X_train_features = prepare_features_with_rules(train_data_encoded, rules)
X_test_features = prepare_features_with_rules(test_data_encoded, rules)

In [ ]:
# Step 6: Reshape data for CNN input
X_train_features = X_train_features.reshape(-1, X_train_features.shape[1], 1, 1)
X_test_features = X_test_features.reshape(-1, X_test_features.shape[1], 1, 1)

In [ ]:
# Step 7: Prepare labels for training
y_train = train_data['label']  # Assuming 'label' is the target column
y_test = test_data['label']
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

In [ ]:
# Step 8: Build the CNN model
model = Sequential()
model.add(Conv2D(32, (1, 1), activation='relu', input_shape=(X_train_features.shape[1], 1, 1)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train_encoded.shape[1], activation='softmax'))

In [ ]:
# Step 9: Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 10: Train the model
model.fit(X_train_features, y_train_encoded, epochs=20, batch_size=32, validation_split=0.1)

In [ ]:
# Step 11: Evaluate the model
loss, accuracy = model.evaluate(X_test_features, y_test_encoded)
print(f'Test Accuracy: {accuracy:.2f}')